In [ ]:
# === colab configuration ===
# p.s. when training on colab, weights are saved on Drive (directory DSCAS/weights).
# p.p.s. skip this cell if running demo file locally!

! pip install torch-summary
! pip install ray
! pip install tensorboardX==2.5.1
! pip install tensorboard==2.11.2
from google.colab import drive, files
import sys

# setting paths
repository_path = '/content/deep-seasonal-color-analysis-system/'
dataset_path = repository_path + 'headsegmentation_dataset_ccncsa/'
dataset_path_drive = '/content/drive/MyDrive/DSCAS/headsegmentation_dataset_ccncsa/'
weights_path = repository_path + 'models/weights/'
weights_path_drive = '/content/drive/MyDrive/DSCAS/weights/'
checkpoint_path = repository_path + 'models/loss_selection/'
checkpoint_archive = '/content/' + checkpoint_path.split('/')[-2] + '.zip'

sys.path.insert(0, repository_path)

# cloning project repository and downloading dataset
drive.mount('/content/drive')
! test ! -d $repository_path && git clone https://github.com/mrcmich/deep-seasonal-color-analysis-system.git
! test ! -d $dataset_path && cp -R $dataset_path_drive $dataset_path
%cd $repository_path

# setting branch and pulling updates
branch = 'main'
! git checkout $branch
! git pull origin $branch

executing_on_colab = True

In [1]:
import os
import torch
from torch import nn, optim
import torchvision.transforms as T
from sklearn.model_selection import train_test_split
from models import dataset, training_and_testing
from models.local.FastSCNN.models import fast_scnn
from metrics_and_losses import metrics
from utils import segmentation_labels, custom_transforms, model_names
from models import config
from slurm_scripts import slurm_config
from ray import tune
from ray.tune import CLIReporter
from functools import partial

try:
  executing_on_colab
except NameError:
  executing_on_colab = False
  weights_path = config.WEIGHTS_PATH
  dataset_path = config.DATASET_PATH
  checkpoint_path = config.LOSS_SELECTION_PATH

In [10]:
# defining transforms
image_transform = T.Compose([T.Resize(slurm_config.GLOBAL_INPUT_SIZE_TRAINING_DEMO), T.Normalize(config.NORMALIZE_MEAN, config.NORMALIZE_STD)])
target_transform = T.Compose([T.Resize(slurm_config.GLOBAL_INPUT_SIZE_TRAINING_DEMO)])

# fetching dataset
n_classes = len(segmentation_labels.labels)
img_paths, label_paths = dataset.get_paths(dataset_path, file_name=config.DATASET_INDEX_NAME)
X_train, _, Y_train, _ = train_test_split(
    img_paths, label_paths, test_size=0.20, random_state=99, shuffle=True)
train_dataset = dataset.CcncsaDataset(X_train, Y_train, image_transform, target_transform)

# model parameters
model_name = "fastscnn"
optimizer = optim.Adam

# setting up hyperparameters for training
device = 'cuda' if torch.cuda.is_available() else 'cpu'
n_epochs = 10
batch_size = 32
score_fn = metrics.batch_mIoU
learning_rate = 0.01
class_weights = torch.tensor(config.CLASS_WEIGHTS, device=device)

# Ray Tune parameters
cpus_per_trial = 0
gpus_per_trial = torch.cuda.device_count()
num_samples = 1  # Number of times each combination is sampled (n_epochs are done per sample)
metric = "val_loss"
metrics_columns = ["train_loss", "train_score", "val_loss", "val_score", "training_iteration"]
local_dir = checkpoint_path + model_names.MODEL_NAMES[model_name]
max_report_frequency = 600
reporter = CLIReporter(
    metric_columns=metrics_columns, max_report_frequency=max_report_frequency)

In [11]:
# === training with std. loss ===

checkpoint_dir = local_dir if executing_on_colab else (os.path.abspath('./' + local_dir) + '/')
cfg = {
    "lr": learning_rate,
    "lr_scheduler": "none",
    "batch_size": batch_size,
    "loss": "std",
    "from_checkpoint": False,
    "checkpoint_dir": checkpoint_dir
    }

model = fast_scnn.FastSCNN(n_classes)
loss_fn = nn.CrossEntropyLoss()

tune.run(partial(training_and_testing.train_model,
                 device=device, model=model, dataset=train_dataset, n_epochs=n_epochs,
                 score_fn=score_fn, loss_fn=loss_fn, optimizer=optimizer, num_workers=(0, 0),
                 evaluate=True, class_weights=class_weights),
         config=cfg,
         num_samples=num_samples,
         resources_per_trial={"cpu": cpus_per_trial, "gpu": gpus_per_trial},
         progress_reporter=reporter,
         checkpoint_at_end=True,
         checkpoint_freq=1,
         local_dir=local_dir)

== Status ==
Current time: 2023-02-02 23:21:29 (running for 00:00:00.39)
Memory usage on this node: 3.8/12.7 GiB 
Using FIFO scheduling algorithm.
Resources requested: 0/2 CPUs, 1.0/1 GPUs, 0.0/7.41 GiB heap, 0.0/3.71 GiB objects
Result logdir: /content/deep-seasonal-color-analysis-system/models/loss_selection/FastSCNN/train_model_2023-02-02_23-21-28
Number of trials: 1/1 (1 RUNNING)
+-------------------------+----------+-------------------+
| Trial name              | status   | loc               |
|-------------------------+----------+-------------------|
| train_model_5222b_00000 | RUNNING  | 172.28.0.12:71783 |
+-------------------------+----------+-------------------+




Trial name,date,done,episodes_total,experiment_id,hostname,iterations_since_restore,node_ip,pid,time_since_restore,time_this_iter_s,time_total_s,timestamp,timesteps_since_restore,timesteps_total,train_loss,train_score,training_iteration,trial_id,val_loss,val_score,warmup_time
train_model_5222b_00000,2023-02-02_23-37-09,False,,3f21c765e05e44e4923a9340f990f7f2,e1d69cf4f5ed,4,172.28.0.12,71783,937.007,232.219,937.007,1675381029,0,,0.188376,0.539164,4,5222b_00000,0.17864,0.563957,0.00322461


== Status ==
Current time: 2023-02-02 23:31:29 (running for 00:10:00.51)
Memory usage on this node: 6.8/12.7 GiB 
Using FIFO scheduling algorithm.
Resources requested: 0/2 CPUs, 1.0/1 GPUs, 0.0/7.41 GiB heap, 0.0/3.71 GiB objects
Result logdir: /content/deep-seasonal-color-analysis-system/models/loss_selection/FastSCNN/train_model_2023-02-02_23-21-28
Number of trials: 1/1 (1 RUNNING)
+-------------------------+----------+-------------------+--------------+---------------+------------+-------------+----------------------+
| Trial name              | status   | loc               |   train_loss |   train_score |   val_loss |   val_score |   training_iteration |
|-------------------------+----------+-------------------+--------------+---------------+------------+-------------+----------------------|
| train_model_5222b_00000 | RUNNING  | 172.28.0.12:71783 |      0.27751 |      0.453381 |   0.253308 |    0.491895 |                    2 |
+-------------------------+----------+---------------

Trial name,date,done,episodes_total,experiment_id,experiment_tag,hostname,iterations_since_restore,node_ip,pid,time_since_restore,time_this_iter_s,time_total_s,timestamp,timesteps_since_restore,timesteps_total,train_loss,train_score,training_iteration,trial_id,val_loss,val_score,warmup_time
train_model_5222b_00000,2023-02-03_00-00-08,True,,3f21c765e05e44e4923a9340f990f7f2,0,e1d69cf4f5ed,10,172.28.0.12,71783,2315.54,228.708,2315.54,1675382408,0,,0.121823,0.616494,10,5222b_00000,0.143779,0.607815,0.00322461


== Status ==
Current time: 2023-02-02 23:41:30 (running for 00:20:01.72)
Memory usage on this node: 6.5/12.7 GiB 
Using FIFO scheduling algorithm.
Resources requested: 0/2 CPUs, 1.0/1 GPUs, 0.0/7.41 GiB heap, 0.0/3.71 GiB objects
Result logdir: /content/deep-seasonal-color-analysis-system/models/loss_selection/FastSCNN/train_model_2023-02-02_23-21-28
Number of trials: 1/1 (1 RUNNING)
+-------------------------+----------+-------------------+--------------+---------------+------------+-------------+----------------------+
| Trial name              | status   | loc               |   train_loss |   train_score |   val_loss |   val_score |   training_iteration |
|-------------------------+----------+-------------------+--------------+---------------+------------+-------------+----------------------|
| train_model_5222b_00000 | RUNNING  | 172.28.0.12:71783 |     0.173217 |      0.556892 |   0.155691 |    0.588691 |                    5 |
+-------------------------+----------+---------------

2023-02-03 00:00:08,299	INFO tune.py:762 -- Total run time: 2319.52 seconds (2319.28 seconds for the tuning loop).


== Status ==
Current time: 2023-02-03 00:00:08 (running for 00:38:39.28)
Memory usage on this node: 6.5/12.7 GiB 
Using FIFO scheduling algorithm.
Resources requested: 0/2 CPUs, 0/1 GPUs, 0.0/7.41 GiB heap, 0.0/3.71 GiB objects
Result logdir: /content/deep-seasonal-color-analysis-system/models/loss_selection/FastSCNN/train_model_2023-02-02_23-21-28
Number of trials: 1/1 (1 TERMINATED)
+-------------------------+------------+-------------------+--------------+---------------+------------+-------------+----------------------+
| Trial name              | status     | loc               |   train_loss |   train_score |   val_loss |   val_score |   training_iteration |
|-------------------------+------------+-------------------+--------------+---------------+------------+-------------+----------------------|
| train_model_5222b_00000 | TERMINATED | 172.28.0.12:71783 |     0.121823 |      0.616494 |   0.143779 |    0.607815 |                   10 |
+-------------------------+------------+----

In [ ]:
# downloading updated training results if running on colab
! if test $executing_on_colab = 'True' ; then zip -r $checkpoint_archive $checkpoint_path ; fi
if executing_on_colab:
    files.download(checkpoint_archive)

In [13]:
# === training with weighted loss ===

cfg["loss"] = "weighted"

model = fast_scnn.FastSCNN(n_classes)
loss_fn = nn.CrossEntropyLoss(weight=class_weights)

tune.run(partial(training_and_testing.train_model,
                 device=device, model=model, dataset=train_dataset, n_epochs=n_epochs,
                 score_fn=score_fn, loss_fn=loss_fn, optimizer=optimizer, num_workers=(0, 0),
                 evaluate=True, class_weights=class_weights),
         config=cfg,
         num_samples=num_samples,
         resources_per_trial={"cpu": cpus_per_trial, "gpu": gpus_per_trial},
         progress_reporter=reporter,
         checkpoint_at_end=True,
         checkpoint_freq=1,
         local_dir=local_dir)

== Status ==
Current time: 2023-02-03 00:02:07 (running for 00:00:00.59)
Memory usage on this node: 3.7/12.7 GiB 
Using FIFO scheduling algorithm.
Resources requested: 0/2 CPUs, 1.0/1 GPUs, 0.0/7.41 GiB heap, 0.0/3.71 GiB objects
Result logdir: /content/deep-seasonal-color-analysis-system/models/loss_selection/FastSCNN/train_model_2023-02-03_00-02-06
Number of trials: 1/1 (1 RUNNING)
+-------------------------+----------+-------------------+
| Trial name              | status   | loc               |
|-------------------------+----------+-------------------|
| train_model_ff19c_00000 | RUNNING  | 172.28.0.12:82187 |
+-------------------------+----------+-------------------+




Trial name,date,done,episodes_total,experiment_id,hostname,iterations_since_restore,node_ip,pid,time_since_restore,time_this_iter_s,time_total_s,timestamp,timesteps_since_restore,timesteps_total,train_loss,train_score,training_iteration,trial_id,val_loss,val_score,warmup_time
train_model_ff19c_00000,2023-02-03_00-14-12,False,,ef0034fa9b924411b4fd366fc606baa3,e1d69cf4f5ed,3,172.28.0.12,82187,716.396,237.706,716.396,1675383252,0,,0.155291,0.514425,3,ff19c_00000,0.160536,0.537647,0.00337434


== Status ==
Current time: 2023-02-03 00:12:09 (running for 00:10:02.99)
Memory usage on this node: 6.8/12.7 GiB 
Using FIFO scheduling algorithm.
Resources requested: 0/2 CPUs, 1.0/1 GPUs, 0.0/7.41 GiB heap, 0.0/3.71 GiB objects
Result logdir: /content/deep-seasonal-color-analysis-system/models/loss_selection/FastSCNN/train_model_2023-02-03_00-02-06
Number of trials: 1/1 (1 RUNNING)
+-------------------------+----------+-------------------+--------------+---------------+------------+-------------+----------------------+
| Trial name              | status   | loc               |   train_loss |   train_score |   val_loss |   val_score |   training_iteration |
|-------------------------+----------+-------------------+--------------+---------------+------------+-------------+----------------------|
| train_model_ff19c_00000 | RUNNING  | 172.28.0.12:82187 |     0.195225 |      0.465376 |    0.17531 |    0.454516 |                    2 |
+-------------------------+----------+---------------

Trial name,date,done,episodes_total,experiment_id,experiment_tag,hostname,iterations_since_restore,node_ip,pid,time_since_restore,time_this_iter_s,time_total_s,timestamp,timesteps_since_restore,timesteps_total,train_loss,train_score,training_iteration,trial_id,val_loss,val_score,warmup_time
train_model_ff19c_00000,2023-02-03_00-41-30,True,,ef0034fa9b924411b4fd366fc606baa3,0,e1d69cf4f5ed,10,172.28.0.12,82187,2354.76,233.112,2354.76,1675384890,0,,0.0831663,0.620111,10,ff19c_00000,0.0842378,0.63575,0.00337434


== Status ==
Current time: 2023-02-03 00:22:14 (running for 00:20:07.98)
Memory usage on this node: 6.5/12.7 GiB 
Using FIFO scheduling algorithm.
Resources requested: 0/2 CPUs, 1.0/1 GPUs, 0.0/7.41 GiB heap, 0.0/3.71 GiB objects
Result logdir: /content/deep-seasonal-color-analysis-system/models/loss_selection/FastSCNN/train_model_2023-02-03_00-02-06
Number of trials: 1/1 (1 RUNNING)
+-------------------------+----------+-------------------+--------------+---------------+------------+-------------+----------------------+
| Trial name              | status   | loc               |   train_loss |   train_score |   val_loss |   val_score |   training_iteration |
|-------------------------+----------+-------------------+--------------+---------------+------------+-------------+----------------------|
| train_model_ff19c_00000 | RUNNING  | 172.28.0.12:82187 |     0.114568 |      0.568347 |   0.128339 |    0.545919 |                    5 |
+-------------------------+----------+---------------

2023-02-03 00:41:30,710	INFO tune.py:762 -- Total run time: 2364.37 seconds (2364.02 seconds for the tuning loop).


== Status ==
Current time: 2023-02-03 00:41:30 (running for 00:39:24.03)
Memory usage on this node: 6.5/12.7 GiB 
Using FIFO scheduling algorithm.
Resources requested: 0/2 CPUs, 0/1 GPUs, 0.0/7.41 GiB heap, 0.0/3.71 GiB objects
Result logdir: /content/deep-seasonal-color-analysis-system/models/loss_selection/FastSCNN/train_model_2023-02-03_00-02-06
Number of trials: 1/1 (1 TERMINATED)
+-------------------------+------------+-------------------+--------------+---------------+------------+-------------+----------------------+
| Trial name              | status     | loc               |   train_loss |   train_score |   val_loss |   val_score |   training_iteration |
|-------------------------+------------+-------------------+--------------+---------------+------------+-------------+----------------------|
| train_model_ff19c_00000 | TERMINATED | 172.28.0.12:82187 |    0.0831663 |      0.620111 |  0.0842378 |     0.63575 |                   10 |
+-------------------------+------------+----

In [ ]:
# downloading updated training results if running on colab
! if test $executing_on_colab = 'True' ; then zip -r $checkpoint_archive $checkpoint_path ; fi
if executing_on_colab:
    files.download(checkpoint_archive)